In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [3]:
df=pd.read_csv("/kaggle/input/vanadium-dioxide-thermochromic/vo2_dataset.csv")

In [4]:
df.isnull().sum()

Power              0
Temperature        0
O2_Ar_Ratio        0
Pressure           0
Target_Material    0
Substrate          0
Label              0
dtype: int64

In [5]:
df


,Power,Temperature,O2_Ar_Ratio,Pressure,Target_Material,Substrate,Label
0,147.454012,679.710838,0.917324,2.048586,V2O5,Al2O3,0
1,205.071431,621.285812,0.122759,0.258108,VO2,NaCa Glass,0
2,183.199394,501.839410,2.001302,2.348379,V,Stainless Steel,0
3,169.865848,520.294309,0.291702,1.876726,V,SiO2,0
4,125.601864,632.700354,0.770559,0.374837,VO2,Si,1
...,...,...,...,...,...,...,...
198,198.708642,680.510581,0.353335,2.695313,V2O3,Al2O3,1
199,187.987555,601.050474,1.067635,1.389808,V2O3,Al2O3,0
200,174.203165,665.291493,0.533301,0.461405,V,Si,1
201,118.413996,564.009920,0.864967,2.858313,V,Stainless Steel,1


In [6]:
df.sample()


,Power,Temperature,O2_Ar_Ratio,Pressure,Target_Material,Substrate,Label
54,169.789998,639.205959,0.346805,0.802374,V2O5,NaCa Glass,1


In [7]:
X=df.drop(columns=["Label"])
X=pd.get_dummies(X).astype(int)
Y=df["Label"]

In [8]:
X

,Power,Temperature,O2_Ar_Ratio,Pressure,Target_Material_V,Target_Material_V2O3,Target_Material_V2O5,Target_Material_VO2,Substrate_Al2O3,Substrate_NaCa Glass,Substrate_Si,Substrate_SiO2,Substrate_Stainless Steel
0,147,679,0,2,0,0,1,0,1,0,0,0,0
1,205,621,0,0,0,0,0,1,0,1,0,0,0
2,183,501,2,2,1,0,0,0,0,0,0,0,1
3,169,520,0,1,1,0,0,0,0,0,0,1,0
4,125,632,0,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,198,680,0,2,0,1,0,0,1,0,0,0,0
199,187,601,1,1,0,1,0,0,1,0,0,0,0
200,174,665,0,0,1,0,0,0,0,0,1,0,0
201,118,564,0,2,1,0,0,0,0,0,0,0,1


In [9]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [10]:
Y_train.unique(
    
)

array([1, 0])

In [11]:
for i in [RandomForestClassifier(),LogisticRegression(),SVC(),GradientBoostingClassifier(),AdaBoostClassifier(),BaggingClassifier(),BernoulliNB()]: 
    random_classif=i
    random_classif.fit(X_train,Y_train)
    print(f"The accuracy of {i}:",random_classif.score(X_test,Y_test))

The accuracy of RandomForestClassifier(): 0.5609756097560976
The accuracy of LogisticRegression(): 0.6097560975609756
The accuracy of SVC(): 0.5853658536585366
The accuracy of GradientBoostingClassifier(): 0.43902439024390244
The accuracy of AdaBoostClassifier(): 0.36585365853658536


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


The accuracy of BaggingClassifier(): 0.4878048780487805
The accuracy of BernoulliNB(): 0.5853658536585366


# # # Quantum deep learning# 

In [12]:
!pip install cirq

In [13]:
import cirq
import tensorflow as tf
from keras.layers import Layer,Dense,Flatten,BatchNormalization,Input,Dropout
from keras.models import Model,Sequential
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.optimizers import Adam


In [14]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(len(data))]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        q=i%len(qubit)
        circuit.append(cirq.H(qubit[q]))
        circuit.append(cirq.rx(value*2*np.pi).on(qubit[q]))
        circuit.append(cirq.H(qubit[q]))
        circuit.append(cirq.X(qubit[q]))
        circuit.append(cirq.ry(value*2*np.pi).on(qubit[q]))
        circuit.append(cirq.Y(qubit[q]))
        circuit.append(cirq.rz(value*2*np.pi).on(qubit[q]))
        circuit.append(cirq.X(qubit[q]))
        circuit.append(cirq.Z(qubit[q]))
        circuit.append(cirq.H(qubit[q]))
        if (i+2)%len(qubit)<=len(qubit):
            circuit.append(cirq.CNOT(qubit[q],qubit[(q+2)%len(qubit)]))
            circuit.append(cirq.CCX(qubit[q],qubit[(q+1)%len(qubit)],qubit[(q+2)%len(qubit )]))
            circuit.append(cirq.measure(qubit[q],key=f"qubit_{i}"))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    for inputs_data in data:
        circuit,qubit=quantum_circuit(inputs_data)
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        measure0=[]
        for i in range(len(inputs_data)):
            measure0.append(result.measurements[f"qubit_{i}"].flatten()[0])
            results.append(measure0)
    return np.array(results,dtype=np.float32)

class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        outputs=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
        outputs.set_shape(inputs.shape)
        return outputs
inputs=Input(shape=(13,))
Q=QuantumLayer()(inputs)
F=Flatten()(Q)
D=Dense(32,activation="relu")(F)
outputs=Dense(1,activation="sigmoid")(D)
model=Model(inputs,outputs)
optimizer=Adam(learning_rate=0.001,beta_1=0.99,beta_2=0.99)
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(X_train, Y_train, batch_size=32, epochs=10)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/binary_crossentropy/logistic_loss/mul/BroadcastGradientArgs defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/pytho

In [15]:
Y_train_reshape=Y_train.values
Y_train_reshape=Y_train_reshape.reshape(-1,1)
Y_train_reshape.shape

(162, 1)

In [24]:
X_train.isnull().sum()

Power                        0
Temperature                  0
O2_Ar_Ratio                  0
Pressure                     0
Target_Material_V            0
Target_Material_V2O3         0
Target_Material_V2O5         0
Target_Material_VO2          0
Substrate_Al2O3              0
Substrate_NaCa Glass         0
Substrate_Si                 0
Substrate_SiO2               0
Substrate_Stainless Steel    0
dtype: int64

In [16]:
X_train.shape

(162, 13)

In [17]:
Y_train.shape

(162,)

In [18]:
import cirq
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Quantum circuit definition
def quantum_circuit(data):
    qubits = [cirq.GridQubit(0, i) for i in range(len(data))]
    circuit = cirq.Circuit()

    for i, value in enumerate(data):
        q = i % len(qubits)
        circuit.append(cirq.H(qubits[q]))
        circuit.append(cirq.rx(value * 2 * np.pi).on(qubits[q]))
        circuit.append(cirq.ry(value * 2 * np.pi).on(qubits[q]))
        circuit.append(cirq.rz(value * 2 * np.pi).on(qubits[q]))
        if (i + 2) % len(qubits) < len(qubits):
            circuit.append(cirq.CNOT(qubits[q], qubits[(q + 1) % len(qubits)]))
        circuit.append(cirq.measure(qubits[q], key=f"qubit_{i}"))

    return circuit, qubits

# Run quantum circuit
def run_quantum_circuit(data):
    results = []
    simulator = cirq.Simulator()

    for input_data in data:
        circuit, qubits = quantum_circuit(input_data)
        result = simulator.run(circuit)
        measure_result = []
        for i in range(len(input_data)):
            measure_result.append(result.measurements[f"qubit_{i}"].flatten()[0])
        results.append(measure_result)

    return np.array(results, dtype=np.float32)

# Define a custom TensorFlow layer for quantum computation
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer, self).__init__()

    def call(self, inputs):
        outputs = tf.numpy_function(run_quantum_circuit, [inputs], tf.float32)
        # Set the shape explicitly based on inputs
        batch_size = inputs.shape[0]
        num_features = inputs.shape[1]
        outputs.set_shape([batch_size, num_features])
        return outputs

# Define the model
inputs = Input(shape=(13,))
quantum_output = QuantumLayer()(inputs)
# flatt = Flatten()(quantum_output)
dense_1 = Dense(32, activation="relu")(quantum_output)
D=Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(dense_1)
D=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)

D=Dense(256,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(556,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(1056,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(2056,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)


outputs = Dense(1, activation="sigmoid")(D)
model = Model(inputs, outputs)

# Compile the model
optimizer = Adam(learning_rate=0.001, beta_1=0.99, beta_2=0.99)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, Y_train, batch_size=32, epochs=10)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 574ms/step - accuracy: 0.5156 - loss: 80.4570
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 555ms/step - accuracy: 0.4548 - loss: 68.8915
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 556ms/step - accuracy: 0.4919 - loss: 58.5829
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 559ms/step - accuracy: 0.5164 - loss: 49.6140
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 555ms/step - accuracy: 0.4131 - loss: 41.8838
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 564ms/step - accuracy: 0.4768 - loss: 35.2425
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 557ms/step - accuracy: 0.4837 - loss: 29.6496
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 552ms/step - accuracy: 0.5135 - loss: 24.9873
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 563ms/step - accuracy: 0.4373 - loss: 21.1681
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 594ms/step - accuracy: 0.5562 - loss: 18.0774


In [19]:
model.predict(X_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 280ms/step


array([[0.48738053],
       [0.4880328 ],
       [0.49905708],
       [0.48804745],
       [0.4968811 ],
       [0.48818508],
       [0.4888105 ],
       [0.4888105 ],
       [0.49344218],
       [0.4949837 ],
       [0.4914118 ],
       [0.49798384],
       [0.47353998],
       [0.4933959 ],
       [0.4867717 ],
       [0.49135718],
       [0.5033921 ],
       [0.47895876],
       [0.4966108 ],
       [0.47973704],
       [0.5079825 ],
       [0.48335528],
       [0.48739338],
       [0.48751843],
       [0.4959014 ],
       [0.51338756],
       [0.4769586 ],
       [0.49296302],
       [0.49851504],
       [0.5033921 ],
       [0.48985887],
       [0.4782319 ],
       [0.49006486],
       [0.48752156],
       [0.49658725],
       [0.4774911 ],
       [0.4848561 ],
       [0.49638832],
       [0.49499658],
       [0.49182555],
       [0.4905431 ]], dtype=float32)

In [20]:
inputs1 = Input(shape=(13,))

dense_1 = Dense(32, activation="relu")(inputs1)
D=Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(dense_1)
D=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)

D=Dense(256,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(556,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(1056,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(2056,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)

# quantum_output = QuantumLayer()(D)
# flattened = Flatten()(quantum_output)

outputs1 = Dense(1, activation="sigmoid")(D)
model1 = Model(inputs1, outputs1)

# Compile the model
optimizer = Adam(learning_rate=0.001, beta_1=0.99, beta_2=0.99)
model1.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model1.fit(X_train, Y_train, batch_size=32, epochs=10)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.5033 - loss: 361.4728
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4496 - loss: 102.2376
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5277 - loss: 75.8698
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4779 - loss: 63.8274
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5325 - loss: 56.8249
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5276 - loss: 50.7486
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5081 - loss: 48.8186
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5285 - loss: 45.5209
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4782 - loss: 42.7176
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5219 - loss: 40.4933


In [23]:
loss,accuray=model1.evaluate(X_test)

ValueError: None values not supported.

In [21]:
model1.predict(X_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


array([[0.27765352],
       [0.05200643],
       [0.09157498],
       [0.11715994],
       [0.23689972],
       [0.08205023],
       [0.20918804],
       [0.08304419],
       [0.07000916],
       [0.23689039],
       [0.03804748],
       [0.06012468],
       [0.09280651],
       [0.13560306],
       [0.23481423],
       [0.33866626],
       [0.11552968],
       [0.27756643],
       [0.05214061],
       [0.30887532],
       [0.06637406],
       [0.08138772],
       [0.09755505],
       [0.06155012],
       [0.07322726],
       [0.24116015],
       [0.27856866],
       [0.04961896],
       [0.13684371],
       [0.2599309 ],
       [0.06744216],
       [0.35244828],
       [0.06677122],
       [0.04461814],
       [0.04728133],
       [0.34989992],
       [0.09083493],
       [0.05369008],
       [0.0783003 ],
       [0.18816261],
       [0.0951411 ]], dtype=float32)

In [32]:
def run_quantum_circuit1(data):
    results = []
    simulator = cirq.Simulator()

    for input_data in data:
        input_data = np.asarray(input_data, dtype=np.float64).flatten()

        circuit, qubits = quantum_circuit(input_data)
        result = simulator.run(circuit)
        measure_result = []
        for i in range(len(input_data)):
            measure_result.append(result.measurements[f"qubit_{i}"].flatten()[0])
        results.append(measure_result)

    return np.array(results, dtype=np.float32)

In [39]:
X_train_quantum = run_quantum_circuit1(X_train)
X_test_quantum = run_quantum_circuit1(X_test)

ValueError: could not convert string to float: 'Power'

In [36]:
X_train["Power"].dtypes

dtype('int64')